## Imports

In [ ]:
!pip install torch numpy pandas networkx matplotlib

import os
import gc
import re
import sys
import abc
import math
import time
import json
import psutil
import shutil
import inspect
import logging
import argparse
import itertools
import subprocess
# comma-separated and wildcard imports are handled!
from typing import *
from enum import Enum, auto
from datetime import datetime
from copy import copy, deepcopy
from functools import partial, lru_cache
from collections import Counter, defaultdict

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split

import numpy as np
import pandas as pd
import networkx as nx
from matplotlib.path import Path
from matplotlib import pyplot as plt

## Utils

---
This is a **Markdown Command** example

_(double-click me to preview the command)_

<!---
Commands have a leading `$` and could be used to amend the generated module tree's structure.
This particular command renames this package from "Utils" to "utilities"

$package=utilities
-->

### File utils

<!-- no commands here, this should be refactored to `./utilities/file_utils.py` -->

In [ ]:
# redundantly importing `os` (on purpose)
import os
# unused import statement (on purpose)
import numpy as np
# we will also randomly use pandas, which was declared in the previous cell

def ensure_dir_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def the_pandas_func():
    # this is just to showcase our CDA at work
    # this module should contain 2 dependencies given our used definitions here
    # `os` and `pandas as pd`
    
    df = pd.DataFrame()
    del df
    
    